**2 задание**

Имеется файл 2 (см. в папке) с различными оценками асессоров.

Формат файла: **login tuid docid jud cjud**.

Пояснение к формату: 

**login** — логин асессора; 

**uid** — id асессора (user id); 

**docid** — id оцениваемого документа (document id); 

**jud** — оценка асессора (judgement); 

**cjud** — правильная оценка (correct judgement); 

разделитель — табуляция \t.

Оценки могут принимать значение [0, 1], т.е. задание, которое сделали асессоры, имеет бинарную шкалу.

Используя данные об оценках, установите, какие асессоры хуже всего справились с заданием. 

На какие показатели вы ориентировались и какие метрики вы использовали для ответа на этот вопрос? 

Можно ли предложить какие-то новые метрики для подсчета качества асессоров с учетом природы оценок у этого бинарного задания?

Опишите подробно все этапы вашего решения.


In [130]:
import pandas as pd
import csv
import numpy as np
from datetime import datetime
from sklearn import metrics

In [131]:
PATH_TO_FILE = "drive/MyDrive/Colab Data/yandex/second.csv"
data = pd.read_csv(PATH_TO_FILE, sep="\t")

In [132]:
data.head(10)

,login,uid,docid,jud,cjud
0,assessor158,158,0,0,0
1,assessor238,238,0,0,0
2,assessor488,488,0,0,0
3,assessor136,136,0,0,0
4,assessor300,300,0,0,0
5,assessor123,123,1,1,1
6,assessor491,491,1,0,1
7,assessor409,409,1,1,1
8,assessor396,396,1,1,1
9,assessor295,295,1,1,1


Для того, чтобы посмотреть на то, как выполняет задания каждый асессор, необходимо сгруппировть нащи данные по логину пользователя. 

Далее я решил посмотреть на стандартые метрики из библиотеки sklern.metrics такие как accuracy, precision и recall. Изначально я подумал, что accuracy будет наиболее приемлемым вариантом для метрики, т.к. нас не столь важнен результат проверки, сколько его правильность, поэтому вполне логичным показалось разделить количество правильных ответов на общее количество ответов - именно это и делает metrics.accuracy_score(response.cjud, response.jud)

In [133]:
data_evaluation = pd.DataFrame()
grouped_data_by_login = data.groupby('login')
data_evaluation['accuracy_score'] = grouped_data_by_login.apply(lambda response: metrics.accuracy_score(response.cjud, response.jud))
data_evaluation['precision_score'] = grouped_data_by_login.apply(lambda response: metrics.precision_score(response.cjud, response.jud))
data_evaluation['recall_score'] = grouped_data_by_login.apply(lambda response: metrics.recall_score(response.cjud, response.jud))


Так же я добавил столбец precision с параметром micro, а всё потому что базовый precision считает только для положительных ответов, что для текущего задания показалось неверным. По итогу эти значения оказались равны accuracy. 

После я добавил столбец precision с параметром macro: он считает точность по каждому классу(в данном случае по [0, 1]), а после берет их среднее. Считаю это наиболее сбалансированной метрикой для нашей задачи.

In [134]:
data_evaluation['precision_score_micro'] = grouped_data_by_login.apply(lambda response: metrics.precision_score(response.cjud, response.jud, average='micro'))
data_evaluation['precision_score_macro'] = grouped_data_by_login.apply(lambda response: metrics.precision_score(response.cjud, response.jud, average='macro'))
data_evaluation['mean_absolute_error'] = grouped_data_by_login.apply(lambda response: metrics.mean_absolute_error(response.cjud, response.jud))

В следующих строчках я решил разделить ответы на четыре группы, где каждое из чисел означает количество соответствующих ответов:

1) tn - правильный ответ 0

2) fp - неправильный ответ 1

3) fn - неправильный ответ 0

4) tp - правильный ответ 1

In [135]:
data_evaluation['confusion_matrix'] = grouped_data_by_login.apply(lambda response: metrics.confusion_matrix(response.cjud, response.jud).ravel())
s = pd.DataFrame(data_evaluation['confusion_matrix'].to_list(), columns=['tn', 'fp', 'fn', 'tp'])
data_evaluation[['tn', 'fp', 'fn', 'tp']] = s[['tn', 'fp', 'fn', 'tp']].to_numpy()

В итоге мне не пригодилось это разбиение, т.к. всё, что я хотел использовать, я нашел в библиотеке sklern.metrics. 

Давайте посмотрим, какие у нас теперь данные

In [136]:
data_evaluation.head()

,accuracy_score,precision_score,recall_score,precision_score_micro,precision_score_macro,mean_absolute_error,confusion_matrix,tn,fp,fn,tp
login,,,,,,,,,,,
assessor0,0.837905,0.453704,0.890909,0.837905,0.716613,0.162095,"[287, 59, 6, 49]",287,59,6,49
assessor1,0.800971,0.344538,0.911111,0.800971,0.665443,0.199029,"[289, 78, 4, 41]",289,78,4,41
assessor10,0.792325,0.275229,0.697674,0.792325,0.618154,0.207675,"[321, 79, 13, 30]",321,79,13,30
assessor100,0.800959,0.339806,0.700000,0.800959,0.646018,0.199041,"[299, 68, 15, 35]",299,68,15,35
assessor101,0.875931,0.445946,0.785714,0.875931,0.709295,0.124069,"[320, 41, 9, 33]",320,41,9,33


Отсортировав данные по столбцу precision_score_micro я могу понять, какие асесссоры имеют низкую точность. Отсортированы данные по возрастанию, поэтому асесссоры с наименьшей точностью находятся ближе к началу.

In [137]:
data_evaluation.sort_values('precision_score_macro').precision_score_macro

login
assessor56     0.483015
assessor3      0.492574
assessor163    0.504060
assessor234    0.510254
assessor390    0.512516
                 ...   
assessor82     0.804985
assessor487    0.808073
assessor462    0.811057
assessor577    0.811441
assessor545    0.819892
Name: precision_score_macro, Length: 600, dtype: float64

В теории ещё можно учитывать сложность задания(то есть как часто в этом задании ощибаются асессоры). Далее учитывать сложные задания меньшим весом.